# Loading the data

First, we are going to load the data that we'll feed to our alorithm in order for it to learn. We have 3 .csv files to import: <br> *x_train.csv*, *x_test.csv* and *y_train.csv*.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from helpers import *

x_train, x_test, y_train, train_ids, test_ids = load_csv_data("/Users/sarazatezalo/Documents/EPFL/semester 1/ML_course/projects/project1/data/dataset_to_release 2")

In [2]:
from preprocessing import preprocess_data
from implementations import *
from utils import *

In [3]:
# Predictions using logistic regression!

gammas = np.logspace(-5, -1, 3)
lambdas = np.logspace(-4, -1, 3)

x_train_pp, x_test_pp, y_train_pp = preprocess_data(x_train, y_train, x_test, neg_label=0)
initial_w = np.zeros((x_train_pp.shape[1],))
k_fold = 5
max_iters = 25

best_lambda, best_gamma, best_acc, best_weights, best_f1 = best_cv_log_reg(y_train_pp, x_train_pp, k_fold, lambdas, gammas, max_iters)

print(f'Best accuracy on {k_fold}-fold: ', best_acc)
print(f'Best F1 score on {k_fold}-fold for class 1 ', best_f1)


Best accuracy on 5-fold:  0.866621969616164
Best F1 score on 5-fold for class 1  0.39011142838043583


In [6]:
print('Best lambda is:', best_lambda)
print('Best gamma is: ', best_gamma)


Best lambda is: 0.0001
Best gamma is:  0.1


In [7]:
# Prediction using best parameters and ridge logistic regression
y_prediction = np.array([-1 if sigmoid(x.T @ best_weights) < 0.5 else 1 for x in x_test_pp])
print(y_prediction.shape)

(109379,)


In [8]:
print('There are this much -1s: ', np.sum((y_prediction==-1).astype(int)))
print('There are this much 1s: ', np.sum((y_prediction==1).astype(int)))


There are this much -1s:  93246
There are this much 1s:  16133


In [18]:
gamma = 0.001
max_iters = 2000
lambda_ = 1e-2
x_train_pp, x_test_pp, y_train_pp = preprocess_data(x_train, y_train, x_test, neg_label=0)
initial_w = np.random.normal(0, 0.01, size=(x_train_pp.shape[1],))
w, loss = reg_logistic_regression(y_train_pp, x_train_pp, lambda_, initial_w, max_iters, gamma)

y_tr_pred = np.array([0 if sigmoid(x.T @ w) < 0.5 else 1 for x in x_train_pp])
print('Accuracy on train test for our weights is: ', accuracy_score(y_tr_pred, y_train_pp))
print('F1 score on train test for our best weights is: ', f1_score(y_tr_pred, y_train_pp, 0))
print('F1 score on train test for our best weights is: ', f1_score(y_tr_pred, y_train_pp))

(328135, 81)
(57950, 81)
Accuracy on train test for our weights is:  0.7264193270060397
F1 score on train test for our best weights is:  0.6733020112100231
F1 score on train test for our best weights is:  0.7646796888915276


In [9]:
create_csv_submission(test_ids, y_prediction, "prediction_17.csv")

In [3]:
x_train_pp, x_test_pp, y_train_pp = preprocess_data(x_train, y_train, x_test, neg_label=-1)
lambdas = np.logspace(-4, 0, 10)
k_fold = 5
best_lambda, best_acc, best_weights, best_f1 = best_cv_ridge_reg(y_train_pp, x_train_pp, k_fold, lambdas)
y_train_prediction = np.array([-1 if x.T @ best_weights < 0 else 1 for x in x_train_pp])

print('Accuracy on KFold for the best model: ', best_acc)
print('F1 on KFold for the best model: ', best_f1)


Accuracy on KFold for the best model:  0.9068188398067868
F1 on KFold for the best model:  0.2784634321651367
